In [1]:
%pylab notebook
#import cetbtools.inspector as ci
import glob
from cetbtools.inspector import make_cetb_geotiff
from netCDF4 import Dataset
import numpy as np
import os
import re

/Users/brodzik/.conda/envs/cetbtools/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


/Users/brodzik/.conda/envs/cetbtools/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [6]:
%cd /Users/brodzik/cetb_swe

/Users/brodzik/cetb_swe


In [7]:
list = glob.glob("*EASE2_N*36H*nc")
list

['NSIDC-0630-EASE2_N25km-AQUA_AMSRE-2003051-36H-E-GRD-RSS-v1.2.nc',
 'NSIDC-0630-EASE2_N3.125km-AQUA_AMSRE-2003051-36H-E-SIR-RSS-v1.2.nc']

In [8]:
for f in list:
    make_cetb_geotiff(f, verbose=True)


cetbtools.inspector:make_cetb_geotiff : TB geotiff image saved to: NSIDC-0630-EASE2_N25km-AQUA_AMSRE-2003051-36H-E-GRD-RSS-v1.2.nc.GRD_TB.tif

cetbtools.inspector:make_cetb_geotiff : TB geotiff image saved to: NSIDC-0630-EASE2_N3.125km-AQUA_AMSRE-2003051-36H-E-SIR-RSS-v1.2.nc.SIR_TB.tif


In [ ]:
import rasterio
with rasterio.open(list[0]) as src:
    data = np.squeeze(src.read())
print(np.amin(data), np.amax(data))

In [ ]:
%cd /Users/brodzik/ecs/DP1/MEASURES/NSIDC-0630.001/2003.02.20
%ls *EASE2_N6.25*AMSRE*18H-E*.nc

In [ ]:
#t36h_file = glob.glob("*N25*AMSRE*36H-E-*nc")
t36h_file = glob.glob("*N3.125*AMSRE*36H-E-*nc")

In [ ]:
print(t36h_file[0])
#print(t36h_file[0])

In [ ]:
def rebin(a, shape):
    sh = shape[0],a.shape[0]//shape[0],shape[1],a.shape[1]//shape[1]
    return a.reshape(sh).mean(-1).mean(1)

In [ ]:
def nsidc1_swe(t36h_file, outDir='/Users/brodzik/cetb_swe/', show=False, bilinear=True):
    
    # Make the corresponding 36h file:
    pattern = re.sub(r'36H', r'18H', t36h_file)
    pattern = re.sub(r'EASE2_N[0-9\.]+km', r'EASE2_N*km', pattern)
    list = glob.glob(pattern)
    t18h_file = list[0]
    outFile = "%s/%s" % (outDir, re.sub(r'36H', r'SWE', t36h_file))
    print('low:  %s' % t18h_file)
    print('high: %s' % t36h_file)
    print('swe:  %s' % outFile)
    
    # Open the files
    f_18h = Dataset(t18h_file, 'r', 'NETCDF4')
    f_36h = Dataset(t36h_file, 'r', 'NETCDF4')
    
    # Read the TB arrays
    t18h = f_18h.variables['TB'][:]
    t36h = f_36h.variables['TB'][:]
    
    # If array sizes don't match, upscale the 18h data
    # or read the bilinear 18h data
    if np.shape(t18h) != np.shape(t36h):
        if bilinear:
            biFile = "%s/%s.SIR_TB.bilinear.tif" % (outDir, t18h_file)
            print("using bilinear file=%s" % biFile)
            with rasterio.open(biFile) as src:
                t18h = src.read()
        else:
            print("Resizing 18H data...")
            t18h = np.repeat(np.repeat(t18h,2,axis=1),2,axis=2)
            
        print("New shapes:")
        print("18h:")
        print(np.shape(t18h))
        print("36h:")
        print(np.shape(t36h))
    
    # Calculate SWE using NSIDC1
    swe = 3. * 1.59 * ((t18h - 6.) - (t36h - 1.))
    swe[swe < 7.5] = 0.
    
    # Start a swe output file, using dimensions/coordinates and crs variable from the 18h file
    dst = Dataset(outFile, "w", "NETCDF4")

    # Copy file attributes
    for name in f_36h.ncattrs():
        dst.setncattr(name, f_36h.getncattr(name))
    
    # Copy dimensions
    for name, dimension in f_36h.dimensions.iteritems():
        dst.createDimension(
            name, (len(dimension) if not dimension.isunlimited else None))
        
    # Copy the coordinate variables and crs variable
    for name, variable in f_36h.variables.iteritems():
        if name in ["TB"]:
            TB_dims = variable.dimensions
        if name in ["time", "y", "x", "crs"]:
            print("Copying %s..." % name)
            outVar = dst.createVariable(name, variable.datatype, variable.dimensions)

            # Copy variable attributes
            outVar.setncatts({k: variable.getncattr(k) for k in variable.ncattrs()})
        
            # Copy the data (crs has no data, only attributes)
            if "crs" != name:
                outVar[:] = f_36h.variables[name][:]
 
    sweVar = dst.createVariable("SWE", swe.dtype, TB_dims)
    sweVar.grid_mapping = "crs"
    sweVar.long_name = "SWE"
    sweVar.units = "mm"
    sweVar.coverage_content_type = "image"
    sweVar.valid_range = [0., 600.]
    sweVar[:] = swe

    dst.close()
    f_18h.close()
    f_36h.close()
    
    print("Wrote SWE to %s" % outFile)
    
    if show:
        fig, ax = plt.subplots(1)
        ax.set_title("SWE")
        ax.imshow(np.squeeze(swe), cmap="Blues_r", vmin=0., vmax=300., interpolation='nearest')
        ax.axis('off')
        plt.tight_layout()

In [ ]:
nsidc1_swe(t36h_file[0], show=True, bilinear=True)

In [ ]:
def nsidc1_swe_at_6(t36h_file, outDir='/Users/brodzik/cetb_swe/', show=False):
    
    # Make the corresponding 36h file:
    pattern = re.sub(r'36H', r'18H', t36h_file)
    pattern = re.sub(r'EASE2_N[0-9\.]+km', r'EASE2_N*km', pattern)
    list = glob.glob(pattern)
    t18h_file = list[0]
    outFile = "%s/%s" % (outDir, re.sub(r'18H', r'SWE', t18h_file))
    print('low:  %s' % t18h_file)
    print('high: %s' % t36h_file)
    print('swe:  %s' % outFile)
    
    # Open the files
    f_18h = Dataset(t18h_file, 'r', 'NETCDF4')
    f_36h = Dataset(t36h_file, 'r', 'NETCDF4')
    
    # Read the TB arrays
    t18h = f_18h.variables['TB'][:]
    t36h = f_36h.variables['TB'][:]
    
    # If array sizes don't match, scale the 3km 36 data to 6.25 km by averaging
    # every 4 cells:
    if np.shape(t18h) != np.shape(t36h):
        print("Resizing 36H data...")
        print(t36h.dtype)
        t36h = np.squeeze(t36h)
        t36h = rebin(t36h, np.shape(t18h)[1:])
        t36h = float32(np.reshape(t36h, np.shape(t18h)))
        print(t36h.dtype)
        print("New shapes:")
        print("18h:")
        print(np.shape(t18h))
        print("36h:")
        print(np.shape(t36h))
    
    # Calculate SWE using NSIDC1
    swe = 3. * 1.59 * ((t18h - 6.) - (t36h - 1.))
    swe[swe < 7.5] = 0.
    
    # Start a swe output file, using dimensions/coordinates and crs variable from the 18h file
    dst = Dataset(outFile, "w", "NETCDF4")

    # Copy file attributes
    for name in f_18h.ncattrs():
        dst.setncattr(name, f_18h.getncattr(name))
    
    # Copy dimensions
    for name, dimension in f_18h.dimensions.iteritems():
        dst.createDimension(
            name, (len(dimension) if not dimension.isunlimited else None))
        
    # Copy the coordinate variables and crs variable
    for name, variable in f_18h.variables.iteritems():
        if name in ["TB"]:
            TB_dims = variable.dimensions
        if name in ["time", "y", "x", "crs"]:
            print("Copying %s..." % name)
            outVar = dst.createVariable(name, variable.datatype, variable.dimensions)

            # Copy variable attributes
            outVar.setncatts({k: variable.getncattr(k) for k in variable.ncattrs()})
        
            # Copy the data (crs has no data, only attributes)
            if "crs" != name:
                outVar[:] = f_18h.variables[name][:]
 
    sweVar = dst.createVariable("SWE", swe.dtype, TB_dims)
    sweVar.grid_mapping = "crs"
    sweVar.long_name = "SWE"
    sweVar.units = "mm"
    sweVar.coverage_content_type = "image"
    sweVar.valid_range = [0., 600.]
    sweVar[:] = swe

    dst.close()
    f_18h.close()
    f_36h.close()
    
    print("Wrote SWE to %s" % outFile)
    
    if show:
        fig, ax = plt.subplots(1)
        ax.set_title("SWE")
        ax.imshow(np.squeeze(swe), cmap="Blues_r", vmin=0., vmax=300., interpolation='nearest')
        ax.axis('off')
        plt.tight_layout()

In [ ]:
nsidc1_swe_at_6(t36h_file[0], show=True)

In [ ]:
dst.close()


In [ ]:
f_18h = Dataset(t18h_file[0], 'r', 'NETCDF4')
f_36h = Dataset(t36h_file[0], 'r', 'NETCDF4')

In [ ]:
#t18h = np.squeeze(f_18h.variables['TB'][:])
#t36h = np.squeeze(f_36h.variables['TB'][:])
t18h = f_18h.variables['TB'][:]
t36h = f_36h.variables['TB'][:]

In [ ]:
print(np.amin(t18h), np.amax(t18h))
print(np.amin(t36h), np.amax(t36h))

In [ ]:
print(np.shape(t18h), np.shape(t36h))

In [ ]:
swe = 3. * 1.59 * ((t18h - 6.) - (t36h - 1.))
swe[swe < 7.5] = 0.

In [ ]:
print(np.shape(swe), np.amin(swe), np.amax(swe))

In [ ]:
fig, ax = plt.subplots(1)
ax.set_title("SWE")
ax.imshow(np.squeeze(swe), cmap="Blues_r", vmin=0., vmax=300., interpolation='nearest')
ax.axis('off')
plt.tight_layout()

In [ ]:
# Start a swe file, using dimensions/coordinates and crs variable from the 18h file
testOutfile = "/Users/brodzik/cetb_swe/test.nc"
dst = Dataset(testOutfile, "w", "NETCDF4")
# Copy file attributes
for name in f_18h.ncattrs():
    dst.setncattr(name, f_18h.getncattr(name))
# Copy dimensions
for name, dimension in f_18h.dimensions.iteritems():
    dst.createDimension(
        name, (len(dimension) if not dimension.isunlimited else None))
# Copy the coordinate variables and crs variable
for name, variable in f_18h.variables.iteritems():
    if name in ["time", "y", "x", "crs", "TB"]:
        print("Copying %s..." % name)
        if name == "TB":
            TB_dims = variable.dimensions

        outName = "%s_18h" % name if name == "TB" else name
        outVar = dst.createVariable(outName, variable.datatype, variable.dimensions)

        # Copy variable attributes
        outVar.setncatts({k: variable.getncattr(k) for k in variable.ncattrs()})
        
        # Copy the data (crs has no data, only attributes)
        if "crs" != name:
            outVar[:] = f_18h.variables[name][:]
 
sweVar = dst.createVariable("SWE", swe.dtype, TB_dims)
sweVar.grid_mapping = "crs"
sweVar.long_name = "SWE"
sweVar.units = "mm"
sweVar.coverage_content_type = "image"
sweVar.valid_range = [0., 600.]
sweVar[:] = swe

dst.close()

In [ ]:
dst.close()
#print(name)
#help(variable)



In [ ]:
f_new = Dataset(testOutfile, 'r', 'NETCDF4')
#swe_copy = f_new.variables['SWE'][:]
#f_new.close()

In [ ]:
f_new

In [ ]:
print(np.shape(swe_copy), np.amin(swe_copy), np.amax(swe_copy))

In [ ]:
fig, ax = plt.subplots(1)
ax.set_title("New SWE")
ax.imshow(np.squeeze(swe_copy), cmap="Blues_r", vmin=0., vmax=300., interpolation='nearest')
ax.axis('off')
plt.tight_layout()

In [ ]:
from cetbtools.inspector import make_cetb_geotiff

In [ ]:
list = glob.glob("/Users/brodzik/cetb_swe/*2003085*SWE*.nc")
list

In [ ]:
for f in list:
    make_cetb_geotiff(f, var_name="SWE", verbose=True)

In [ ]:
avg_tb = get_avg_tb(filenames)
avg_tb.shape, np.amin(avg_tb), np.amax(avg_tb)

In [ ]:
fig, ax = plt.subplots(1, figsize=(xdim, ydim))
ax.set_title("viridis")
ax.imshow(avg_tb, cmap="viridis", vmin=140., vmax=320, interpolation='nearest')
ax.axis('off')
plt.tight_layout()


In [ ]:
test = np.reshape(np.arange(16),[4,4])
print(test)

In [ ]:
new = np.repeat(np.repeat(test,2,axis=0),2,axis=1)
new

In [ ]:
def rebin(a, shape):
    sh = shape[0],a.shape[0]//shape[0],shape[1],a.shape[1]//shape[1]
    return a.reshape(sh).mean(-1).mean(1)


In [ ]:
rebin(test,(2,2))

In [ ]:
np.shape(test)

In [ ]:
new = np.reshape(test, (1,4,4))

In [ ]:
np.shape(new)[1:]

In [ ]:
float32(new)

In [ ]:
tb[2880,:].data

In [ ]:
new

In [ ]:
my_copy

In [ ]:
other_copy = new.copy()

In [ ]:
new

In [ ]:
other_copy

In [ ]:
other_copy[1,1] = 6.

In [ ]:
new

In [ ]:
other_copy

In [ ]:
other_copy = other_copy + new
other_copy

In [ ]:
arr

In [ ]:
arr[arr < 1.] = 1.
arr

In [ ]:
other_copy

In [ ]:
other_copy / arr